In [59]:
df = pd.read_csv('CoDNaS entries - Compiled list.csv')

In [72]:
import pandas as pd
import requests
from io import StringIO
from pathlib import Path





















  0%|▏                                                                                                                                                      | 5/3524 [00:16<01:28, 39.54it/s]

In [60]:
df

,ID_POOL_CoDNaS,UniProt,#CONF,RMSD min,RMSD max,RMSD avg,Protein Name,Taxon ID,139122
0,1A11_A,P25110,16,0,2.97,1.088,ACETYLCHOLINE RECEPTOR M2,10116,39.478434
1,1A1T_A,Q75677,70,0.56,14.56,6.5689,NUCLEOCAPSID PROTEIN,11676,NaN
2,1A1W_A,Q13158,2,0.97,0.97,0.97,FADD PROTEIN,9606,NaN
3,1A66_A,O95644,28,0.85,9.01,5.2691,CORE NFATC1,9606,NaN
4,1A6S_A,P03322,120,0,9.52,1.7949,GAG POLYPROTEIN,11888,NaN
...,...,...,...,...,...,...,...,...,...
3519,6UH1_C,A0A0E3SXU7,7,0.28,0.81,0.5348,VP3,39054,NaN
3520,6Z3Q_D,Q6YLE3,6,0.15,1.27,0.708,VP4,39054,NaN
3521,7A69_C,no_data,7,0.28,1.19,0.8614,MRK16 Fab-fragment heavy chain,10090,NaN
3522,7C9W_E,P08174,3,0.18,1.26,0.89,Complement decay-accelerating factor,9606,NaN


In [78]:
def downlaod_codnas_cluster(codnas_id):
    if Path(f'codnas_raw_data/{codnas_id}.csv').is_file():
        return
    print(codnas_id)
    
    try:
        r = requests.post("http://ufq.unq.edu.ar/codnas/php/download_builder.php", data={
            "ids_download":codnas_id, 
            "Mammoth_RMS":"1",
            "Causes_DC":"1"
        })
        entry = pd.read_csv(StringIO(r.text), sep="\t")
        entry.to_csv(f'codnas_raw_data/{codnas_id}.csv')
    except Exception:
        with open(f'codnas_raw_data/{codnas_id}.csv.err', "w") as f:
            pass

In [44]:
from joblib import Parallel, delayed
import multiprocessing
from tqdm import tqdm

In [117]:
Parallel(n_jobs=5)(delayed(downlaod_codnas_cluster)(codnas_id) for codnas_id in tqdm(df.ID_POOL_CoDNaS))















  0%|                                                                                                                                                               | 0/3524 [00:00<?, ?it/s]













  0%|▏                                                                                                                                                      | 5/3524 [00:00<01:44, 33.57it/s]













  0%|▍                                                                                                                                                     | 10/3524 [00:00<02:44, 21.37it/s]













100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 3524/3524 [00:00<00:00, 6408.70it/s]













 41%|███████████████████████████████████████████████████████████▉                                                                                        | 1385/3417 [00:38<00

2PNI_A
2PPD_A
2PQI_A
2PSP_A
2PST_X
2R35_A
4YEO_A
6VB2_B
1HTV_B
5WB2_B
5WBT_A
5WH1_A
5WLB_A
5WSG_T


KeyboardInterrupt: 

In [116]:
import biotite.database.rcsb as rcsb
from biotite.structure.io.pdb import PDBFile
from pathlib import Path
import pandas as pd
from biotite.structure import get_residue_starts

In [128]:
def parse_group(codnas_group_file):
    #print(codnas_group_file)
    group_name = Path(codnas_group_file).stem
    #print("Running", group_name)
    df = pd.read_csv(codnas_group_file, index_col=0)

    if "Mammoth_RMS" not in df.columns:
        return
        
    df = df[pd.to_numeric(df['Mammoth_RMS'], errors='coerce').notnull()]

    df["Mammoth_RMS"] = df["Mammoth_RMS"].astype(float)
    
    if len(df[df.Mammoth_RMS>4])==0:
        return

    try:
        df = df.groupby(pd.cut(df["Mammoth_RMS"], np.arange(0, df["Mammoth_RMS"].max(), df["Mammoth_RMS"].quantile(0.2)))).head(4)
    except ValueError:
        return
    

    try:
        df = df.assign(
            pdb1=df["PDB_1"].str[:4],
            pdb2=df["PDB_2"].str[:4],
            model1=df["PDB_1"].astype(str).apply(lambda s: s.split("-")[1].split("_")[0] if "-" in s else -1).astype(int),
            model2=df["PDB_2"].astype(str).apply(lambda s: s.split("-")[1].split("_")[0] if "-" in s else -1).astype(int),
            chain1=df["PDB_1"].str.split("_", expand=True)[1],
            chain2=df["PDB_2"].str.split("_", expand=True)[1],
        )
    except (TypeError, KeyError):
        print(codnas_group_file)
        print(df)
        return

    #print(df)
        
    proteins = set(p for grp in df.groupby(["pdb1", "pdb2"]).groups.keys() for p in grp)
    for p in proteins:
        #print(p)
        atoms_stack = None
        

        pdb_df = df[(df.pdb1==p)|(df.pdb2==p)]

        models = set(pdb_df[["model1", "model2"]].values.flatten())
        for m_id in models:
            #print(m_id)
            atom_array = None
            
            chains = set(pdb_df[["chain1", "chain2"]].values.flatten())
            for ch_id in chains:
                #print(ch_id)
                new_path = Path(f'codnas_groups_pdb/{group_name}/{p}-{m_id}-{ch_id}.pdb')
                if new_path.is_file():
                    #print("Skipping, exists")
                    continue
                
                if atoms_stack is None:
                    pdb_filename = Path("raw_pdbs")/ f"{p.upper()}.pdb"
                    if not pdb_filename.is_file():
                        try:
                            rcsb.fetch(p, "pdb", 'raw_pdbs')
                        except Exception:
                            continue
                    try:
                        pdb_file = PDBFile.read(str(pdb_filename))
                        atoms_stack = pdb_file.get_structure(extra_fields=['atom_id', 'b_factor', 'occupancy'])
                    except Exception as e:
                        #print("Skipping", e)
                        continue
                if atom_array is None:
                    try:
                        if m_id != -1:
                            atom_array = atoms_stack[m_id-1]
                        else:
                            atom_array = atoms_stack[0]
                    except IndexError as e:
                        #print("Skipping atom array", e)
                        continue

                #print("Loaded atom_array")
                
                chain_array = atom_array[atom_array.chain_id==ch_id]
                if len(get_residue_starts(chain_array))>200:
                    #print("Skipping")
                    continue
                new_file = PDBFile()
                try:
                    new_file.set_structure(chain_array)
                except TypeError:
                    return
                new_path.parent.mkdir(exist_ok=True, parents=True)
                new_file.write(str(new_path))
                #print("Saving", new_path)


In [ ]:
from joblib import Parallel, delayed
import multiprocessing
from tqdm import tqdm
import numpy as np

In [129]:
Parallel(n_jobs=-1)(delayed(parse_group)(codnas_group_file) \
                    for codnas_group_file in tqdm(list(Path('codnas_raw_data').glob("*.csv"))))

























  0%|                                                                                                                                                               | 0/3427 [00:00<?, ?it/s]






















  7%|███████████                                                                                                                                          | 255/3427 [00:08<01:46, 29.88it/s]






















 15%|██████████████████████▏                                                                                                                              | 510/3427 [00:08<00:42, 67.91it/s]/tmp/ipykernel_3197881/1786413909.py:18: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
/tmp/ipykernel_3197881/1786413909.py:18: FutureWarning: The default of observed=False i

[None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,

In [131]:
from pdbfixer import PDBFixer
from openmm.app.pdbfile import PDBFile as mmPDBFile

In [111]:
!pwd

/home/bournelab/qbi_hackathon























 57%|████████████████████████████████████████████████████████████████████████████████████▎                                                               | 1947/3417 [00:58<00:19, 74.26it/s]

In [132]:
def fixandwritePDB(name):
    new_file = Path(name).parent / f"{name.stem}_fixed.pdb"
    if new_file.is_file():
        return
    
    new_file.parent.mkdir(exist_ok=True, parents=True)
    
    if name.stat().st_size < 100:
        return
    try:
        fixer = PDBFixer(filename=str(name))
        fixer.findMissingResidues()
        fixer.findNonstandardResidues()
        fixer.replaceNonstandardResidues()
        fixer.removeHeterogens(True)
        fixer.findMissingAtoms()
        fixer.addMissingAtoms()
    except Exception:
        return
        
    
    with new_file.open("w") as f:
        mmPDBFile.writeFile(fixer.topology, fixer.positions, f)

In [133]:
Parallel(n_jobs=-1)(delayed(fixandwritePDB)(f) for f in tqdm(list(Path('codnas_groups_pdb').glob("**/*.pdb"))))
























  0%|                                                                                                                                                              | 0/19202 [00:00<?, ?it/s]






















  3%|███▉                                                                                                                                               | 510/19202 [00:03<02:02, 152.43it/s]






















  4%|█████▉                                                                                                                                              | 765/19202 [00:14<06:38, 46.22it/s]






















  4%|█████▉                                                                                                                                              | 765/19202 [00:24<06:38, 46.22it/s]






















  5%|███████▋                                                                                                                    

codnas_raw_data/1AJ7_L.csv
Empty DataFrame
Columns: [PDB_1, PDB_2, Pool_repr, Mammoth_RMS, Causes_DC, Unnamed: 5]
Index: []

























 11%|███████████████▍                                                                                                                                 | 2040/19202 [08:05<1:36:18,  2.97it/s]

codnas_raw_data/4OZT_U.csv
Empty DataFrame
Columns: [PDB_1, PDB_2, Pool_repr, Mammoth_RMS, Causes_DC, Unnamed: 5]
Index: []
codnas_raw_data/1OO3_A.csv
Empty DataFrame
Columns: [PDB_1, PDB_2, Pool_repr, Mammoth_RMS, Causes_DC, Unnamed: 5]
Index: []

























 12%|█████████████████▎                                                                                                                               | 2295/19202 [09:25<1:33:03,  3.03it/s]

...

codnas_raw_data/1F2H_A.csv
Empty DataFrame
Columns: [PDB_1, PDB_2, Pool_repr, Mammoth_RMS, Causes_DC, Unnamed: 5]
Index: []
codnas_raw_data/1FU5_A.csv
Empty DataFrame
Columns: [PDB_1, PDB_2, Pool_repr, Mammoth_RMS, Causes_DC, Unnamed: 5]
Index: []
codnas_raw_data/1DOT_A.csv
Empty DataFrame
Columns: [PDB_1, PDB_2, Pool_repr, Mammoth_RMS, Causes_DC, Unnamed: 5]
Index: []

























 15%|█████████████████████▏                                                                                                                           | 2805/19202 [14:47<2:08:36,  2.13it/s]

codnas_raw_data/1M8L_A.csv
Empty DataFrame
Columns: [PDB_1, PDB_2, Pool_repr, Mammoth_RMS, Causes_DC, Unnamed: 5]
Index: []

























 16%|███████████████████████                                                                                                                          | 3060/19202 [15:03<1:33:35,  2.87it/s]






















 16%|███████████████████████                                                                                                                          | 3060/19202 [15:14<1:33:35,  2.87it/s]






















 17%|█████████████████████████                                                                                                                        | 3315/19202 [18:21<2:06:23,  2.09it/s]






















 19%|██████████████████████████▉                                                                                                                      | 3570/19202 [19:26<1:46:45,  2.44it/s]






















 20%|████████████████████████████▉                                                                                               

[None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,

In [8]:
from pathlib import Path